In [1]:
import codecs
import networkx as nx
import os
import pickle
from networkx.drawing.nx_pydot import write_dot
from os import listdir
from os.path import isfile, join

In [2]:
vertex_labels = pickle.load(open('label_vertex.p', 'rb'))
edge_labels = pickle.load(open('label_edge.p', 'rb'))

In [6]:
joined_result_dir = 'joined_results'
result_file_name = 'result_gSpan-s50_l3.txt'
params = result_file_name.split('-')[1].replace('.txt', '')
out_names = list()
with codecs.open(result_file_name, 'r', encoding='utf-8') as f:
    res = ''
    members = ''
    for line in f:
        res += line
        if line[0] == '-':
            f_out = codecs.open(join(joined_result_dir, params + '_' + str(out_name).rjust(4, '0') + '.txt'), 'a+', encoding='utf-8')
            f_out.write(res)
            f_out.close()
            res = ''
        elif line[0] == 'w':
            members = line.replace('\r\n', '').replace(' ', '').replace('[', '').replace(']', '').split(':')[1]
            try:
                out_name = out_names.index(members)
            except:
                out_name = len(out_names)
                out_names.append(members)

In [7]:
input_files = [f for f in listdir(joined_result_dir) if isfile(join(joined_result_dir, f))]
temp_out_dir = 'graphs_from_gSpan'
for input_file in input_files:
    H = nx.DiGraph()
    print(input_file)
    with codecs.open(join(joined_result_dir, input_file), 'r', encoding='utf-8') as f:
        for line in f:
            if line[0] == 't':
                result_number = line.split()[2]
                vertex_label_indices = list()
            elif line[0] == 'v':
                vertex_label_indices.append(int(line.split()[2]))
            elif line[0] == 'e':
                line_elements = line.split()
                v1 = vertex_label_indices[int(line_elements[1])]
                v2 = vertex_label_indices[int(line_elements[2])]
                H.add_edge(vertex_labels[v1], vertex_labels[v2], label=edge_labels[int(line_elements[3])])
            elif line[0] == 'w':
                members = line.replace('\r\n', '').replace(' ', '').replace('[', '').replace(']', '').split(':')[1]
            elif line[0] == '-':
                f_out = codecs.open(join(temp_out_dir, input_file.replace('.txt', '.members.txt')), 'w')
                member_list = members.split(',')
                member_list = list(map(int, member_list))
                member_list.sort()
                for member in member_list:
                    f_out.write('F' + str(member).rjust(5, '0') + '\n')
                f_out.close()
                write_dot(H, join(temp_out_dir, input_file.replace('.txt', '.dot')))
    H.clear()

s50_l3_0000.txt
s50_l3_0001.txt
s50_l3_0002.txt
s50_l3_0003.txt
s50_l3_0004.txt
s50_l3_0005.txt
s50_l3_0006.txt
s50_l3_0007.txt
s50_l3_0008.txt
s50_l3_0009.txt
s50_l3_0010.txt
s50_l3_0011.txt
s50_l3_0012.txt
s50_l3_0013.txt
s50_l3_0014.txt
s50_l3_0015.txt
s50_l3_0016.txt
s50_l3_0017.txt
s50_l3_0018.txt
s50_l3_0019.txt
s50_l3_0020.txt
s50_l3_0021.txt
s50_l3_0022.txt
s50_l3_0023.txt
s50_l3_0024.txt
s50_l3_0025.txt
s50_l3_0026.txt
s50_l3_0027.txt
s50_l3_0028.txt
s50_l3_0029.txt
s50_l3_0030.txt
s50_l3_0031.txt
s50_l3_0032.txt
s50_l3_0033.txt
s50_l3_0034.txt
s50_l3_0035.txt
s50_l3_0036.txt
s50_l3_0037.txt
s50_l3_0038.txt
s50_l3_0039.txt
s50_l3_0040.txt
s50_l3_0041.txt
s50_l3_0042.txt
s50_l3_0043.txt
s50_l3_0044.txt
s50_l3_0045.txt
s50_l3_0046.txt
s50_l3_0047.txt
s50_l3_0048.txt
s50_l3_0049.txt
s50_l3_0050.txt
s50_l3_0051.txt
s50_l3_0052.txt
s50_l3_0053.txt
s50_l3_0054.txt
s50_l3_0055.txt
s50_l3_0056.txt
s50_l3_0057.txt
s50_l3_0058.txt
s50_l3_0059.txt
s50_l3_0060.txt
s50_l3_0061.txt
s50_l3_0